<a href="https://colab.research.google.com/github/drigger091/RAG_CHATBOT/blob/main/Fine_tune_Transformers_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
pip install transformers -U

In [6]:
data = pd.read_csv("/content/train.csv")

In [7]:
import pandas as pd

In [8]:
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [9]:
data['toxic'].value_counts()

toxic
0    144277
1     15294
Name: count, dtype: int64

0 represnts not toxic
1 represent toxic

In [10]:
data = data[['comment_text','toxic']]
data = data[0:5000]   # limiting t 1000 lines
data.head()

,comment_text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [11]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score,precision_score, recall_score
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer


In [12]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
model = model.to('cuda')
sample_data = ['i am playing','i am drinking']
tokenizer(sample_data, padding=True, truncation=True, max_length=512)

{'input_ids': [[101, 1045, 2572, 2652, 102], [101, 1045, 2572, 5948, 102]], 'token_type_ids': [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}

In [18]:
# traiing and spliting data

X = list(data['comment_text'])
y = list(data['toxic'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

In [15]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [19]:
len(X_train), len(X_test)


(3500, 1500)

In [29]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [30]:
train_dataset = Dataset(X_train_tokenized, y_train)
test_dataset = Dataset(X_test_tokenized, y_test)

In [31]:
train_dataset[5]

{'input_ids': tensor([  101,  1045,  1005,  1049,  3374,  1045,  1005,  1049,  3374,  1045,
         14180,  2105,  2007,  2619,  2015,  2831,  3931,  1012,  2009,  2001,
          2200,  2919,  2000,  2079,  1012,  1045,  2113,  2129,  2383,  1996,
         23561,  2015,  2006,  2037,  2831,  3931,  7126,  2017, 20865,  2115,
         13811,  2058,  2068,  1012,  1045,  2113,  1045,  2323,  6812,  2091,
          2000,  1996, 26668, 15631,  1012,  2021,  2059,  2153,  1010,  1045,
          1005,  1049,  2183,  2000,  2175,  2377,  2648,  1012,  1012,  1012,
          1012,  2007,  2115,  3566,  1012,  6146,  1012, 13092,  1012,  6535,
          1012,  6445,   102,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [32]:
def compute_metrics(p):
  print(type(p))
  pred, labels = p
  pred = np.argmax(pred, axis=1)
  accuracy = accuracy_score(y_true=labels, y_pred=pred)
  f1 = f1_score(y_true=labels, y_pred=pred)
  precision = precision_score(y_true=labels, y_pred=pred)
  recall = recall_score(y_true=labels, y_pred=pred)

  return {"accuracy": accuracy, "f1": f1, "precision": precision, "recall": recall}

In [33]:
# definng the trainer
args = TrainingArguments(
    output_dir = "output",
    num_train_epochs = 10,
    per_device_train_batch_size = 16
)

trainer = Trainer(
    model = model,
    args = args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    compute_metrics = compute_metrics
)

In [34]:
trainer.train()

Step,Training Loss
500,0.129000
1000,0.010100
1500,0.002300
2000,0.000000


TrainOutput(global_step=2190, training_loss=0.03230730255546908, metrics={'train_runtime': 3319.1193, 'train_samples_per_second': 10.545, 'train_steps_per_second': 0.66, 'total_flos': 9208886937600000.0, 'train_loss': 0.03230730255546908, 'epoch': 10.0})

In [35]:
   trainer.evaluate()

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.2999928295612335,
 'eval_accuracy': 0.9626666666666667,
 'eval_f1': 0.8181818181818182,
 'eval_precision': 0.8456375838926175,
 'eval_recall': 0.7924528301886793,
 'eval_runtime': 41.7262,
 'eval_samples_per_second': 35.949,
 'eval_steps_per_second': 4.506,
 'epoch': 10.0}

In [36]:
np.set_printoptions(suppress=True)

In [37]:
trainer.save_model('FineTuneModel')

In [38]:
 model_2 = BertForSequenceClassification.from_pretrained("/content/FineTuneModel")
 model_2.to('cuda')

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [49]:
# text = "That was good point"
text = "excellent job done"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model_2(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = predictions.cpu().detach().numpy()
first_element = predictions[0][0]
print(f"First element: {first_element:.10f}")
second_element = predictions[0][1]
print(f"Second element: {second_element:.10f}")

if first_element > second_element:
  print(" the comment is not toxic")
else:
  print("The comment is toxic")


First element: 0.9999824762
Second element: 0.0000175516
 the comment is not toxic
